In [4]:
!pip install opencage

  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: certifi
    Found existing installation: certifi 2023.7.22
    Uninstalling certifi-2023.7.22:
      Successfully uninstalled certifi-2023.7.22

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [25]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
api_key = '01265c10f13344ba856290ffed95facc'
geocoder = OpenCageGeocode(api_key)
input_file = '/Users/yaminiane/Documents/Hackathon/Air Quality/Transformed Data/Public Schools.xlsx'
df = pd.read_excel(input_file)
address_columns = ['Street Address', 'City', 'State', 'ZIP']
print(df)
print(df.shape)

      NCES School ID State School ID  NCES District ID State District ID  \
0       180004602162    IN-9545-4164           1800046           IN-9545   
1       180657000433    IN-4925-4792           1806570           IN-4925   
2       180003000001    IN-0125-0045           1800030           IN-0125   
3       180288000362    IN-4670-3945           1802880           IN-4670   
4       180432000730    IN-4710-4463           1804320           IN-4710   
...              ...             ...               ...               ...   
1912    180732001232    IN-1970-1524           1807320           IN-1970   
1913    180283000341    IN-0630-0512           1802830           IN-0630   
1914    180283000342    IN-0630-0511           1802830           IN-0630   
1915    180283001551    IN-0630-0517           1802830           IN-0630   
1916    180283002046    IN-0630-0510           1802830           IN-0630   

     Low Grade* High Grade*                        School Name  \
0            KG      

In [30]:
def get_geolocation(address):
    query = f"{address['PSS_ADDRESS']}, {address['PSS_CITY']}, {address['PSS_STABB']} {address['PSS_ZIP5']}, USA"
    results = geocoder.geocode(query)
    
    if results:
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        return lat, lng
    else:
        return None, None
        
def get_batch_geolocations(batch):
    return [get_geolocation(row) for _, row in batch.iterrows()]

def process_geolocation_parallel(df, batch_size=10):
    batches = [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(get_batch_geolocations, batches))

    flattened_results = [item for sublist in results for item in sublist]
    latitudes, longitudes = zip(*flattened_results)

    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    print(latitudes, longitudes)

In [27]:
output_file = 'PublicSchoolLocations.xlsx'
df.to_excel(output_file, index=False)

In [ ]:
36af303a98e34c60b2c731efa3e43c47

In [52]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
api_key = 'd20152d437c5446983855fb2d61221eb'
geocoder = OpenCageGeocode(api_key)
input_file = '/Users/yaminiane/Documents/Hackathon/Code Files/HospitalLocations.xlsx'
df = pd.read_excel(input_file)
address_columns = ['Site Address', 'Site City', 'State Name',	'Site Postal Code']
print(df)
print(df.shape)

                             Health Center Type Health Center Number  \
0      Federally Qualified Health Center (FQHC)           H80CS00563   
1      Federally Qualified Health Center (FQHC)           H80CS00205   
2      Federally Qualified Health Center (FQHC)           H80CS10748   
3      Federally Qualified Health Center (FQHC)           H80CS00811   
4      Federally Qualified Health Center (FQHC)           H80CS28356   
...                                         ...                  ...   
18204  Federally Qualified Health Center (FQHC)           H80CS00867   
18205  Federally Qualified Health Center (FQHC)           H80CS00341   
18206  Federally Qualified Health Center (FQHC)           H80CS00858   
18207  Federally Qualified Health Center (FQHC)           H80CS12868   
18208  Federally Qualified Health Center (FQHC)           H80CS00768   

      BHCMIS Organization Identification Number BPHC Assigned Number  \
0                                        033130       BPS-H80-0

In [53]:
df= df[df['State Name'] == 'Indiana']

In [54]:
def get_geolocation(address):
    query = f"{address['Site Address']}, {address['Site City']}, {address['State Name']}, {address['Site Postal Code']}, USA"
    results = geocoder.geocode(query)
    
    if results:
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        return lat, lng
    else:
        return None, None
        
def get_batch_geolocations(batch):
    return [get_geolocation(row) for _, row in batch.iterrows()]

def process_geolocation_parallel(df, batch_size=10):
    batches = [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(get_batch_geolocations, batches))

    flattened_results = [item for sublist in results for item in sublist]
    latitudes, longitudes = zip(*flattened_results)

    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    print(latitudes, longitudes)

In [55]:
df.shape

(452, 55)

In [56]:
process_geolocation_parallel(df,10)

(39.8213469, 39.781506, 40.869905, 39.784434, 41.6071332, 41.6462824, 38.6430427, 41.5527315, 41.4114158, 40.282028, 39.7745204, 41.545071, 40.20527, 40.1915513, 41.629404, 41.3476075, 40.4394844, 41.6819694, 39.6764549, 38.6894731, 41.6136166, 38.6553352, 41.5440596, 41.6418824, 38.3801803, 38.6744855, 39.9224536, 37.9775048, 38.289041, 39.83088, 39.7991496, 41.6361049, 41.5591171, 40.194804, 38.436311, 39.7884583, 41.6020878, 41.6016789, 39.8082861, 39.77439, 39.1595304, 41.646092, 39.1661098, 40.425684, 41.489949, 38.436311, 39.758743, 39.4652263, 39.5757265, 39.827132, 41.550535, 39.4777996, 39.810967, 39.7635478, 40.5317716, 39.6847886, 40.7563089, 39.8426979, 38.2981343, 41.478108, 40.047627, 41.611555, 40.2625768, 40.096894, 41.6290221, 39.825768, 41.3736543, 41.5660178, 40.902317, 41.3766395, 40.0954125, 39.8118915, 39.802892, 40.1720927, 39.810321, 40.16014, 39.8154779, 40.5598268, 41.5590145, 40.256154, 39.7878331, 39.735169, 39.794573, 41.537364, 40.480025, 41.200314, 41.475

In [57]:
output_file = 'HospitalLocations.xlsx'
df.to_excel(output_file, index=False)